In [ ]:
import csv, time, requests, json, datetime
import hmac
import hashlib
import sys
!{sys.executable} -m pip install websocket-client
import websocket 
import ssl

webSocket ='wss://stream.binance.com:9443/ws/xvgbtc@ticker'

ws = websocket.WebSocket(sslopt={"cert_reqs": ssl.CERT_NONE})
connect = ws.connect(webSocket)
binTick = ws.recv()
binPrice = json.loads(binTick)

def calculate_signature(secret, data=None, params=None):
    ordered_data = data
    query_string = '&'.join(["{}={}".format(d[0], d[1]) for d in ordered_data])
    m = hmac.new(bytes(secret, 'latin-1'), query_string.encode('utf-8'), hashlib.sha256)
    return m.hexdigest()

MY_API_KEY = ''
CLIENT_SECRET = ''

timeStamp = []
totalArray = []
orderUrl = 'https://api.binance.com/api/v3/order'
accountUrl = 'https://api.binance.com/api/v3/account'
tradeUrl = 'https://api.binance.com/api/v3/myTrades'

headers = {
    'X-MBX-APIKEY': MY_API_KEY,
}

btcAccount = .0011
buyPrice = 0
sellPrice = 0

buy = False
sell = False

buyOrder = []
sellOrder = []
iBuy = 0
iSell = 0
while(True):
    binTick = ws.recv()
    binPrice = json.loads(binTick)
    
    currentTargetBid = float(binPrice["b"])
    currentTargetAsk = float(binPrice["a"])
    
    if(buy==False):
        buyPrice = currentTargetBid
        data = [
            ('symbol', binPrice["s"]),
            ('side', 'BUY'),
            ('type', 'LIMIT'),
            ('timeInForce', 'GTC'),   
            ('quantity', '{:.8f}'.format(int(btcAccount/(currentTargetBid-0.00000001)))),
            ('price', '{:.8f}'.format(currentTargetBid-0.00000001)),   
            ('recvWindow', '6000000'),
            ('timestamp', binPrice["E"]),
        ]
        data.append(
            ('signature', calculate_signature(CLIENT_SECRET, data=data)))
        r1 = requests.post(orderUrl, data=data, headers=headers)
        r1Text = r1.text
        print(r1Text)
        r1ID = json.loads(r1Text)
        try:
            buyOrder.append(r1ID["orderId"])
            print("here1")
            buy = True
            iBuy =  iBuy + 1
        except KeyError:
            z = 0
            while(z < iBuy):
                data = [
                    ('symbol', binPrice["s"]),
                    ('orderId', buyOrder[z]),
                    ('timestamp', binPrice["E"]),
                    ('recvWindow', '6000000'),
                ]
                data.append(
                    ('signature', calculate_signature(CLIENT_SECRET, data=data)))
                p1 = requests.delete(orderUrl, data=data, headers=headers)
                print(p1.text)
                z = z + 1
            buyOrder = []
            iBuy = 0
        
    if(sell==False):
        sellPrice = currentTargetAsk
        data = [
            ('symbol', binPrice["s"]),
            ('side', 'SELL'),
            ('type', 'LIMIT'),
            ('timeInForce', 'GTC'),   
            ('quantity', '{:.8f}'.format(int(btcAccount/(currentTargetAsk+0.00000001)))),
            ('price', '{:.8f}'.format(currentTargetAsk+0.00000001)),   
            ('recvWindow', '6000000'),
            ('timestamp', binPrice["E"]),
        ]
        data.append(
            ('signature', calculate_signature(CLIENT_SECRET, data=data)))
        r2 = requests.post(orderUrl, data=data, headers=headers)
        r2Text = r2.text
        print(r2Text)
        r2ID = json.loads(r2Text)
        try:
            sellOrder.append(r2ID["orderId"])
            print("here2")
            sell = True
            iSell =  iSell + 1
        except KeyError:
            z = 0
            while(z < iSell):
                data = [
                    ('symbol', binPrice["s"]),
                    ('orderId', sellOrder[z]),
                    ('timestamp', binPrice["E"]),
                    ('recvWindow', '6000000'),
                ]
                data.append(
                    ('signature', calculate_signature(CLIENT_SECRET, data=data)))
                p1 = requests.delete(orderUrl, data=data, headers=headers)
                print(p1.text)
                z = z + 1
            sellOrder = []
            iSell = 0
        
    if(currentTargetBid > buyPrice):
        sell = False
        buy = False
    if(currentTargetAsk < sellPrice):
        buy = False
        sell = False

ws.close()
